### 9. Spark SQL

In this notebook we will look at Spark SQL. Spark SQL lets us work on structured data. The notebook will follow the contents of [Spark Documentation](https://spark.apache.org/docs/2.2.1/sql-programming-guide.html). With the extra information Spark has for the structure of the data, some additional optimizations can be performed.


#### Datasets and Dataframes

Dataset is a distrbuted collection of data. Dataset can be constructed from JVM objects and comes with the benefits of Strong typing and ability to use lambda functions of RDDs along with the optimization advantages of Spark SQL.

Dataframe is conceptually equivalent of a database table and has richer optimization under the hood. In Scala ``DataFrame`` is an alias of ``Dataset[Row]``

In [9]:
val spark = new org.apache.spark.sql.SQLContext(sc).sparkSession

First, we create an instance of ``org.apache.spark.sql.SparkSession`` from the available ``sc`` object of ``org.apache.spark.SparkContext``. To that that we need to instantiate ``org.apache.spark.sql.SQLContext`` from with the current ``sc`` variable and then get the ``sparkSession``

In [15]:
val peopledf = spark.read.json("people.json")
peopledf.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



What we saw above is loaded froma JSON file with contents 

```
{"name":"Michael"}
{"name":"Andy", "age":30}
{"name":"Justin", "age":19}

```

The file was downloaded from [this](https://github.com/apache/spark/blob/master/examples/src/main/resources/people.json) URL. The ``SparkSession`` instance was used to create a ``DataFrame`` instance from this JSON file and an appropriate type was inferred on loading the content. To view the schema of the `Dataframe` we do the following and we see that age is a numeric type and name is a string.

In [22]:
peopledf.printSchema

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)




All operations on ``DataFrame`` are untyped and they dont fail until we execute them. We will see some examples below of perfectly valid operations

In [34]:
import spark.implicits._

peopledf.select("name").show()
peopledf.select($"name", $"age" + 1).show()
peopledf.filter($"age" > 20).show()
peopledf.groupBy($"age").count().show()


+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
+-------+

+-------+---------+
|   name|(age + 1)|
+-------+---------+
|Michael|     null|
|   Andy|       31|
| Justin|       20|
+-------+---------+

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+

+----+-----+
| age|count|
+----+-----+
|  19|    1|
|null|    1|
|  30|    1|
+----+-----+



By importing ``import spark.implicits._`` we get to use ``$``. This is a convenient way to convert a string to a type ``org.apache.spark.sql.ColumnName``. The following two piece code fragments give the same results

``peopledf.filter(new org.apache.spark.sql.ColumnName("age") > 20).show()``

and

``peopledf.filter($"age" > 20).show()``

no prize for guessing which is more readable, especially if we want to specify multiple column names.

The ``select`` and ``filter`` operations returns us another``DataFrame``, however ``groupBy`` returns an object ``org.apache.spark.sql.RelationalGroupedDataset`` which further provides more aggregation operations like ``mean``, ``min``, ``max``, ``sum``, ``count``, ``pivot`` etc. 

The operations on ``DataFrames`` is not type safe and allows us to perfrom operations on types which doesn't make sense and gives unexpected results or even errors at run time. Following two examples, first where we compare a string value to be greater than a number giving us no results and second, we select a non existant field throwing a runtime exception.

In [54]:
peopledf.filter($"name" > 20).show()
peopledf.select($"test").show()

+---+----+
|age|name|
+---+----+
+---+----+



Name: org.apache.spark.sql.AnalysisException
Message: cannot resolve '`test`' given input columns: [age, name];;
'Project ['test]
+- AnalysisBarrier
      +- Relation[age#26L,name#27] json

StackTrace: 'Project ['test]
+- AnalysisBarrier
      +- Relation[age#26L,name#27] json

  at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:42)
  at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$2.applyOrElse(CheckAnalysis.scala:88)
  at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$2.applyOrElse(CheckAnalysis.scala:85)
  at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$transformUp$1.apply(TreeNode.scala:289)
  at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$transformUp$1.apply(TreeNode.scala:289)
  at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:70)
  at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(Tree



#### Running SQL Queries

We can run sql like queries on ``DataFrames`` as follows. We can register a dataframe as a view either in the session or a global temporart view which can be used to query the data as follows.

Notive how we prefix the ``global_temp.`` to the name of the view like we do for a schema of a table in a relational DB

In [59]:
//A view local to session
peopledf.createOrReplaceTempView("PeopleTemp")

//A  global temp view
peopledf.createOrReplaceGlobalTempView("PeopleGlobal")

spark.sql("select * from PeopleTemp").show()

spark.sql("select * from global_temp.PeopleGlobal").show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+




We will now see how to create ``Dataset`` We do the following 

- Create a case class
- Convert a sequence of case class instance to a ``Dataset``
- show the contents of a ``Dataset``
- Print the schema of the ``Dataset``

In [64]:
case class Person(name: String, age: Long)

val caseClassDS = Seq(Person("Andy", 32)).toDS()

caseClassDS.show()

caseClassDS.printSchema

+----+---+
|name|age|
+----+---+
|Andy| 32|
+----+---+

root
 |-- name: string (nullable = true)
 |-- age: long (nullable = false)





A ``DataFrame`` can be converted to a ``Dataset`` as follows



In [68]:
val personDS = spark.read.json("people.json").as[Person]

personDS.show()

personDS.printSchema

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)





We will see how we can interoperate between SQL and programatic API of Spark. Notice though that when we use ``DataFrames`` we no longer have strong typing and need to know the structure of the data when we use the programatic approach. The show function accepts a parameter ``truncate`` and we set it to false to show the entire content of the column.

In [82]:

val teenageDF = spark.sql("select * from PeopleTemp where age between 13 and 19")

teenageDF.show()
//Map names

teenageDF.map(teenage => "Name: " + teenage(1) + ", Age: " + teenage(0)).show(truncate = false)

+---+------+
|age|  name|
+---+------+
| 19|Justin|
+---+------+

+--------------------+
|value               |
+--------------------+
|Name: Justin,Age: 19|
+--------------------+





### Aggregations

We will now see how we can perform aggregations on ``DataFrame`` and ``Dataset``


For these examples we will use ``employees.json`` downloaded from [thus](https://raw.githubusercontent.com/apache/spark/master/examples/src/main/resources/employees.json) URL. 

First we will create a ``DataFrame`` from thus data file and then find the mean as follows

In [90]:
val employeeDF = spark.read.json("employees.json")
employeeDF.show()
employeeDF.printSchema
employeeDF.createOrReplaceTempView("Employees")
spark.sql("select mean(salary) as MeanSalary from Employees").show()

+-------+------+
|   name|salary|
+-------+------+
|Michael|  3000|
|   Andy|  4500|
| Justin|  3500|
|  Berta|  4000|
+-------+------+

root
 |-- name: string (nullable = true)
 |-- salary: long (nullable = true)

+----------+
|MeanSalary|
+----------+
|    3750.0|
+----------+




But what if we want to write our own aggregation function that we want to use? This is similar to the aggregate or reduce call we will make on an RDD but is bit complicated. We will see step by step what we need to do 

- create an object that extends from ``org.apache.spark.sql.expressions.UserDefinedAggregateFunction``
- There are several abstract methods we need to implement. We will see what they mean and what they are. As an example we will implement the same mean function we will call it ``myMean``.

    - We start with what is the input to the function. In our case of myMean, the input type is a number.We then start by implementing the method ``inputSchema`` which returns a field ``StructType`` which tells us the data type of the input field(s). In our case it will be one field of the ``DataFrame`` and that frame will be of type ``Long``    
    - Then there is a buffer, which will hold the values when the aggregation is being done. In case of ``myMean`` we need to hold two values, the running total and the number of rows we visited. We thus implement our next method ``bufferSchema`` which will again return a ``StructType``, but this time the returned datatype has two fields, both of type Long to hold the running total and running count.
    - There is another datatype which is the datatype of the return type of the ``myMean`` function. In this case it will be Long type of data and this is represented by yet another ``StructType`` and is implemented by the field ``dataType``.
    - There is a method called ``deterministic`` which we implement to return true or false which essentially tells us if the value always same for a given set of inputs. This information probably is used by Spark to use cached values in case the given input was already seen and computed and the value is deterministic.
    - We then initialize the buffer to the starting value. In our case for mean, both running total and number of records processed is set to 0. The method to be implemented is called ``initialize``
    - We need to update the buffer with a new record. In case if ``myMean`` we simply increment the record count in the buffer by one and add the running total with the value we have for the current record. This is done by implenting the ``update`` method.
    - The ``DataFrame`` is distributed and the aggregation happens on various partitions in parallel. There needs to be a way to merge two buffers into one. In our case of we simply add the running total and number of records processed from both buffers in one buffer. This is done by implementing the method ``merge``.
    - Finally we need to evaluate the final value using the accumulated buffer. In our case, we divide the  running total with the total number of records. This is done by implementing the ``evaluate`` function.


Once this object is implemented, simply register it with spark session as a udf (Use Defined Function) with a given name. Once this is done we can start using it in the query functions like any other inbuilt function like mean.

Lets see how we implement this ``myMean`` in Spark.

TODO: Show the implementation